In [9]:
# Install required packages
!pip install xgboost lightgbm catboost optuna torch --quiet
print("Packages installed!")

Packages installed!


In [2]:
# Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import time
import gc
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import KFold, cross_val_score, GroupKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.linear_model import Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Gradient Boosting
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# Hyperparameter Tuning
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, OneCycleLR

# Set seeds
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Using device: cuda
GPU: NVIDIA A100-SXM4-40GB
Memory: 42.4 GB


In [3]:
# Load data
train_features = pd.read_csv('train_hh_features.csv')
train_gt = pd.read_csv('train_hh_gt.csv')
train_rates = pd.read_csv('train_rates_gt.csv')
test_features = pd.read_csv('test_hh_features.csv')

# Merge
train_df = train_features.merge(train_gt[['hhid', 'cons_ppp17']], on='hhid', how='left')

print(f"Training: {train_df.shape[0]:,} households")
print(f"Test: {test_features.shape[0]:,} households")
print(f"Train surveys: {sorted(train_df['survey_id'].unique())}")
print(f"Test surveys: {sorted(test_features['survey_id'].unique())}")

Training: 104,234 households
Test: 103,023 households
Train surveys: [np.int64(100000), np.int64(200000), np.int64(300000)]
Test surveys: [np.int64(400000), np.int64(500000), np.int64(600000)]


In [4]:
def preprocess_and_engineer_features(df, is_train=True):
    """Preprocess data and create engineered features."""
    df = df.copy()
    
    # Binary Yes/No columns (food consumption)
    binary_yes_no_cols = [c for c in df.columns if c.startswith('consumed')]
    for col in binary_yes_no_cols:
        if df[col].dtype == 'object':
            df[col] = (df[col] == 'Yes').astype(int)
    
    # Male/Female
    if 'male' in df.columns and df['male'].dtype == 'object':
        df['male'] = (df['male'] == 'Male').astype(int)
    
    # Access/No access columns
    access_cols = ['water', 'toilet', 'sewer', 'elect']
    for col in access_cols:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = (df[col] == 'Access').astype(int)
    
    # Owner
    if 'owner' in df.columns and df['owner'].dtype == 'object':
        df['owner'] = (df['owner'] == 'Owner').astype(int)
    
    # Urban
    if 'urban' in df.columns and df['urban'].dtype == 'object':
        df['urban'] = (df['urban'] == 'Urban').astype(int)
    
    # Employed
    if 'employed' in df.columns and df['employed'].dtype == 'object':
        df['employed'] = df['employed'].map({'Employed': 1, 'Not employed': 0}).fillna(0).astype(int)
    
    # any_nonagric
    if 'any_nonagric' in df.columns and df['any_nonagric'].dtype == 'object':
        df['any_nonagric'] = (df['any_nonagric'] == 'Yes').astype(int)
    
    # Label encode remaining categorical columns
    cat_cols_to_encode = ['water_source', 'sanitation_source', 'dweltyp', 'educ_max', 'sector1d']
    for col in cat_cols_to_encode:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].fillna('Unknown')
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
    
    # HOUSEHOLD COMPOSITION FEATURES
    df['total_children'] = df['num_children5'] + df['num_children10'] + df['num_children18']
    df['total_adults'] = df['num_adult_female'] + df['num_adult_male']
    df['dependency_ratio'] = (df['total_children'] + df['num_elderly']) / (df['total_adults'] + 0.1)
    df['children_ratio'] = df['total_children'] / (df['hsize'] + 0.1)
    df['elderly_ratio'] = df['num_elderly'] / (df['hsize'] + 0.1)
    df['adults_ratio'] = df['total_adults'] / (df['hsize'] + 0.1)
    df['female_adult_ratio'] = df['num_adult_female'] / (df['total_adults'] + 0.1)
    
    # FOOD CONSUMPTION AGGREGATES
    food_cols = [c for c in df.columns if c.startswith('consumed')]
    df['food_diversity'] = df[food_cols].sum(axis=1)
    df['food_diversity_ratio'] = df['food_diversity'] / len(food_cols)
    
    basic_foods = [f'consumed{i}' for i in [100, 200, 300, 400, 500, 600, 700] if f'consumed{i}' in df.columns]
    df['basic_food_count'] = df[basic_foods].sum(axis=1) if basic_foods else 0
    
    protein_foods = [f'consumed{i}' for i in [700, 800, 900, 1000, 1100, 1200, 2000, 2100, 2200] if f'consumed{i}' in df.columns]
    df['protein_food_count'] = df[protein_foods].sum(axis=1) if protein_foods else 0
    
    luxury_foods = [c for c in food_cols if int(c.replace('consumed', '')) >= 3000]
    df['luxury_food_count'] = df[luxury_foods].sum(axis=1) if luxury_foods else 0
    
    df['food_wealth_ratio'] = df['luxury_food_count'] / (df['basic_food_count'] + 1)
    
    # HOUSING QUALITY INDEX
    housing_vars = ['elect', 'water', 'toilet', 'sewer', 'owner']
    available_housing = [v for v in housing_vars if v in df.columns]
    df['housing_quality'] = df[available_housing].sum(axis=1)
    
    # EMPLOYMENT FEATURES
    if 'sworkershh' in df.columns:
        df['high_employment'] = (df['sworkershh'] > 0.5).astype(int)
    if 'sfworkershh' in df.columns:
        df['has_formal_work'] = (df['sfworkershh'] > 0).astype(int)
    
    # INTERACTION FEATURES
    if 'urban' in df.columns:
        df['urban_housing'] = df['urban'] * df['housing_quality']
    df['food_per_capita'] = df['food_diversity'] / (df['hsize'] + 0.1)
    if 'employed' in df.columns:
        df['age_employed'] = df['age'] * df['employed']
    
    # PER CAPITA FEATURES
    if 'utl_exp_ppp17' in df.columns:
        df['utl_per_capita'] = df['utl_exp_ppp17'] / (df['hsize'] + 0.1)
        df['log_utl_exp'] = np.log1p(df['utl_exp_ppp17'])
    
    # ADDITIONAL ADVANCED FEATURES
    df['hsize_squared'] = df['hsize'] ** 2
    df['age_squared'] = df['age'] ** 2
    df['log_hsize'] = np.log1p(df['hsize'])
    
    # Region interactions
    region_cols = [c for c in df.columns if c.startswith('region')]
    df['num_regions'] = df[region_cols].sum(axis=1)
    
    return df

# Apply preprocessing
print("Preprocessing...")
train_df = preprocess_and_engineer_features(train_df)
test_df = preprocess_and_engineer_features(test_features)
print(f"Features: {train_df.shape[1]}")

Preprocessing...
Features: 114


In [5]:
# Prepare feature matrix
exclude_cols = ['hhid', 'survey_id', 'cons_ppp17', 'com', 'strata']
feature_cols = [c for c in train_df.columns if c not in exclude_cols]
common_cols = [c for c in feature_cols if c in test_df.columns]

X = train_df[common_cols].fillna(train_df[common_cols].median()).values
y = train_df['cons_ppp17'].values
y_log = np.log1p(y)
groups = train_df['survey_id'].values

X_test = test_df[common_cols].fillna(train_df[common_cols].median()).values

# Scaling
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

print(f"X: {X.shape}, X_test: {X_test.shape}")

X: (104234, 109), X_test: (103023, 109)


In [6]:
def weighted_mape(y_true, y_pred):
    """Calculate Weighted Mean Absolute Percentage Error."""
    mask = y_true != 0
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / np.abs(y_true[mask])) * 100

def leave_one_survey_out_cv(model_fn, X, y, groups, use_log=True):
    """Leave-One-Survey-Out CV with model factory function."""
    unique_surveys = np.unique(groups)
    cv_scores = []
    oof_preds = np.zeros(len(y))
    
    for test_survey in unique_surveys:
        train_mask = groups != test_survey
        test_mask = groups == test_survey
        
        X_train, X_val = X[train_mask], X[test_mask]
        
        if use_log:
            y_train = np.log1p(y[train_mask])
            y_val = y[test_mask]
        else:
            y_train, y_val = y[train_mask], y[test_mask]
        
        model = model_fn()
        model.fit(X_train, y_train)
        
        if use_log:
            y_pred = np.expm1(model.predict(X_val))
        else:
            y_pred = model.predict(X_val)
        
        y_pred = np.clip(y_pred, y.min() * 0.5, y.max() * 1.5)
        oof_preds[test_mask] = y_pred
        
        score = weighted_mape(y_val, y_pred)
        cv_scores.append(score)
    
    return np.mean(cv_scores), cv_scores, oof_preds

In [7]:
def objective_xgb(trial):
    """Optuna objective for XGBoost."""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 1500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'device': 'cuda',        # Ενεργοποιεί την GPU
        'tree_method': 'hist',   # Ο γρήγορος αλγόριθμος για GPU
        'random_state': SEED,
        'n_jobs': -1
    }
    
    def model_fn():
        return xgb.XGBRegressor(**params)
    
    score, _, _ = leave_one_survey_out_cv(model_fn, X, y, groups)
    return score

def objective_lgb(trial):
    """Optuna objective for LightGBM."""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 1500),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': SEED,
        'n_jobs': -1,
        'verbosity': -1
    }
    
    def model_fn():
        return lgb.LGBMRegressor(**params)
    
    score, _, _ = leave_one_survey_out_cv(model_fn, X, y, groups)
    return score

In [13]:
# Run Optuna optimization for XGBoost
print("="*60)
print("OPTUNA OPTIMIZATION - XGBoost")
print("="*60)

sampler = TPESampler(seed=SEED)
study_xgb = optuna.create_study(direction='minimize', sampler=sampler)
study_xgb.optimize(objective_xgb, n_trials=50, show_progress_bar=True)

print(f"\nBest XGBoost wMAPE: {study_xgb.best_value:.4f}%")
print(f"Best params: {study_xgb.best_params}")

best_xgb_params = study_xgb.best_params

OPTUNA OPTIMIZATION - XGBoost


  0%|          | 0/50 [00:00<?, ?it/s]


Best XGBoost wMAPE: 27.8692%
Best params: {'n_estimators': 1298, 'max_depth': 8, 'learning_rate': 0.014708461075644571, 'subsample': 0.7983441751135245, 'colsample_bytree': 0.6472052324149987, 'reg_alpha': 1.1682929741329003e-06, 'reg_lambda': 1.1747830124068715e-08, 'min_child_weight': 9}


In [14]:
# Run Optuna optimization for LightGBM
print("="*60)
print("OPTUNA OPTIMIZATION - LightGBM")
print("="*60)

study_lgb = optuna.create_study(direction='minimize', sampler=sampler)
study_lgb.optimize(objective_lgb, n_trials=50, show_progress_bar=True)

print(f"\nBest LightGBM wMAPE: {study_lgb.best_value:.4f}%")
print(f"Best params: {study_lgb.best_params}")

best_lgb_params = study_lgb.best_params

OPTUNA OPTIMIZATION - LightGBM


  0%|          | 0/50 [00:00<?, ?it/s]


Best LightGBM wMAPE: 27.8520%
Best params: {'n_estimators': 1474, 'max_depth': 12, 'learning_rate': 0.013875834161346563, 'subsample': 0.9992393683603316, 'colsample_bytree': 0.6051678897664073, 'reg_alpha': 3.463669462971936e-07, 'reg_lambda': 0.08156379676483906, 'num_leaves': 136, 'min_child_samples': 95}


In [8]:
class ResidualBlock(nn.Module):
    """Residual block with skip connection."""
    def __init__(self, in_dim, out_dim, dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, out_dim)
        self.bn1 = nn.BatchNorm1d(out_dim)
        self.fc2 = nn.Linear(out_dim, out_dim)
        self.bn2 = nn.BatchNorm1d(out_dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.SiLU()  # Better than ReLU
        
        # Skip connection
        self.skip = nn.Linear(in_dim, out_dim) if in_dim != out_dim else nn.Identity()
        
    def forward(self, x):
        identity = self.skip(x)
        
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.activation(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        out = self.bn2(out)
        
        out = out + identity  # Residual connection
        out = self.activation(out)
        
        return out


class AdvancedPovertyNet(nn.Module):
    """Advanced Neural Network with Residual Connections."""
    
    def __init__(self, input_dim, hidden_dims=[512, 256, 128, 64], dropout=0.3):
        super().__init__()
        
        # Input projection
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.BatchNorm1d(hidden_dims[0]),
            nn.SiLU(),
            nn.Dropout(dropout)
        )
        
        # Residual blocks
        self.res_blocks = nn.ModuleList()
        for i in range(len(hidden_dims) - 1):
            self.res_blocks.append(
                ResidualBlock(hidden_dims[i], hidden_dims[i+1], dropout)
            )
        
        # Output
        self.output = nn.Sequential(
            nn.Linear(hidden_dims[-1], 32),
            nn.SiLU(),
            nn.Linear(32, 1)
        )
        
    def forward(self, x):
        x = self.input_proj(x)
        
        for block in self.res_blocks:
            x = block(x)
        
        return self.output(x).squeeze()


class AdvancedNNWrapper:
    """Advanced Neural Network wrapper with better training."""
    
    def __init__(self, input_dim, hidden_dims=[512, 256, 128, 64], 
                 epochs=200, batch_size=512, lr=0.001, patience=20, dropout=0.3):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.epochs = epochs
        self.batch_size = batch_size
        self.lr = lr
        self.patience = patience
        self.dropout = dropout
        self.device = device
        self.model = None
        self.best_model_state = None
        
    def fit(self, X, y):
        X_tensor = torch.FloatTensor(X).to(self.device)
        y_tensor = torch.FloatTensor(y).to(self.device)
        
        self.model = AdvancedPovertyNet(
            self.input_dim, self.hidden_dims, self.dropout
        ).to(self.device)
        
        optimizer = optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=1e-4)
        criterion = nn.HuberLoss(delta=1.0)  # More robust than MSE
        
        # Cosine annealing with warm restarts
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2)
        
        dataset = TensorDataset(X_tensor, y_tensor)
        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        
        best_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(self.epochs):
            self.model.train()
            epoch_loss = 0
            
            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                
                optimizer.step()
                epoch_loss += loss.item()
            
            scheduler.step()
            avg_loss = epoch_loss / len(loader)
            
            if avg_loss < best_loss:
                best_loss = avg_loss
                self.best_model_state = self.model.state_dict().copy()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= self.patience:
                    break
        
        # Load best model
        if self.best_model_state:
            self.model.load_state_dict(self.best_model_state)
        
        return self
    
    def predict(self, X):
        self.model.eval()
        X_tensor = torch.FloatTensor(X).to(self.device)
        
        with torch.no_grad():
            predictions = self.model(X_tensor).cpu().numpy()
        
        return predictions

In [9]:
# Test Advanced Neural Network
print("="*60)
print("ADVANCED NEURAL NETWORK")
print("="*60)

def nn_model_fn():
    return AdvancedNNWrapper(
        input_dim=X_scaled.shape[1],
        hidden_dims=[512, 256, 128, 64],
        epochs=200,
        batch_size=512,
        lr=0.001,
        patience=25,
        dropout=0.35
    )

nn_score, nn_cv_scores, nn_oof = leave_one_survey_out_cv(
    nn_model_fn, X_scaled, y, groups, use_log=True
)

for i, (survey, score) in enumerate(zip(np.unique(groups), nn_cv_scores)):
    print(f"  Survey {survey}: wMAPE = {score:.4f}%")
print(f"\nAdvanced NN - Mean CV wMAPE: {nn_score:.4f}%")

ADVANCED NEURAL NETWORK
  Survey 100000: wMAPE = 35.6565%
  Survey 200000: wMAPE = 31.0237%
  Survey 300000: wMAPE = 33.5767%

Advanced NN - Mean CV wMAPE: 33.4190%


In [10]:
def train_multi_seed(model_class, params, X_train, y_train, X_test, seeds=[42, 123, 456, 789, 2024], use_log=True):
    """Train model with multiple seeds and average predictions."""
    all_preds = []
    
    y_train_use = np.log1p(y_train) if use_log else y_train
    
    for seed in seeds:
        params_copy = params.copy()
        params_copy['random_state'] = seed
        
        model = model_class(**params_copy)
        model.fit(X_train, y_train_use)
        
        preds = model.predict(X_test)
        if use_log:
            preds = np.expm1(preds)
        
        all_preds.append(preds)
    
    return np.mean(all_preds, axis=0)

In [15]:
# Train final models with best parameters
print("="*60)
print("TRAINING FINAL MODELS")
print("="*60)

seeds = [42, 123, 456, 789, 2024]

# XGBoost with optimized params
print("\n1. Training XGBoost (5 seeds)...")
xgb_params = best_xgb_params.copy()
xgb_params['n_jobs'] = -1
xgb_preds = train_multi_seed(xgb.XGBRegressor, xgb_params, X, y, X_test, seeds)
print(f"   Done! Preds range: [{xgb_preds.min():.2f}, {xgb_preds.max():.2f}]")

# LightGBM with optimized params
print("\n2. Training LightGBM (5 seeds)...")
lgb_params = best_lgb_params.copy()
lgb_params['n_jobs'] = -1
lgb_params['verbosity'] = -1
lgb_preds = train_multi_seed(lgb.LGBMRegressor, lgb_params, X, y, X_test, seeds)
print(f"   Done! Preds range: [{lgb_preds.min():.2f}, {lgb_preds.max():.2f}]")

# CatBoost
print("\n3. Training CatBoost (5 seeds)...")
cat_params = {
    'iterations': 800,
    'depth': 6,
    'learning_rate': 0.05,
    'l2_leaf_reg': 3,
    'verbose': 0,
    'task_type': 'GPU',      # Λέει στο CatBoost να πάρει την A100
    'devices': '0'           # Χρήση της πρώτης GPU
}
cat_preds = train_multi_seed(CatBoostRegressor, cat_params, X, y, X_test, seeds)
print(f"   Done! Preds range: [{cat_preds.min():.2f}, {cat_preds.max():.2f}]")

# Neural Network (single training, already averaged internally)
print("\n4. Training Advanced Neural Network...")
nn_final = AdvancedNNWrapper(
    input_dim=X_scaled.shape[1],
    hidden_dims=[512, 256, 128, 64],
    epochs=200,
    batch_size=512,
    lr=0.001,
    patience=25,
    dropout=0.35
)
nn_final.fit(X_scaled, np.log1p(y))
nn_preds = np.expm1(nn_final.predict(X_test_scaled))
print(f"   Done! Preds range: [{nn_preds.min():.2f}, {nn_preds.max():.2f}]") 

TRAINING FINAL MODELS

1. Training XGBoost (5 seeds)...
   Done! Preds range: [1.69, 116.82]

2. Training LightGBM (5 seeds)...
   Done! Preds range: [1.74, 106.64]

3. Training CatBoost (5 seeds)...
   Done! Preds range: [1.71, 115.48]

4. Training Advanced Neural Network...
   Done! Preds range: [1.32, 128.62]


In [16]:
 # Weighted ensemble based on CV performance
# Adjust weights - give more to better performers
weights = {
    'XGBoost': 0.35,
    'LightGBM': 0.35,
    'CatBoost': 0.20,
    'NN': 0.10
}

ensemble_pred = (
    weights['XGBoost'] * xgb_preds +
    weights['LightGBM'] * lgb_preds +
    weights['CatBoost'] * cat_preds +
    weights['NN'] * nn_preds
)

# Clip to reasonable range
ensemble_pred = np.clip(ensemble_pred, y.min() * 0.5, y.max() * 1.5)

print(f"\nFinal Ensemble Predictions:")
print(f"  Range: [{ensemble_pred.min():.2f}, {ensemble_pred.max():.2f}]")
print(f"  Mean: {ensemble_pred.mean():.2f}")
print(f"  Training mean: {y.mean():.2f}")


Final Ensemble Predictions:
  Range: [1.74, 108.80]
  Mean: 11.85
  Training mean: 11.56


In [17]:
# Generate submission files
print("="*60)
print("GENERATING SUBMISSION FILES")
print("="*60)

# Get poverty thresholds
rate_cols = [c for c in train_rates.columns if c.startswith('pct_hh_below')]
poverty_thresholds = [float(c.replace('pct_hh_below_', '')) for c in rate_cols]

def calculate_poverty_rates(predictions, thresholds):
    rates = {}
    n = len(predictions)
    for threshold in thresholds:
        col_name = f'pct_hh_below_{threshold:.2f}'
        rates[col_name] = (predictions < threshold).sum() / n
    return rates

# 1. Household consumption predictions
submission_hh = pd.DataFrame({
    'hhid': test_df['hhid'],
    'cons_ppp17': ensemble_pred
})
print(f"\n1. Household Consumption:")
print(f"   Shape: {submission_hh.shape}")
print(submission_hh.head())

# 2. Poverty rates for EACH survey
test_surveys = test_df['survey_id'].unique()
print(f"\n2. Poverty Rates for {len(test_surveys)} surveys:")

submission_rates_list = []
for survey_id in test_surveys:
    survey_mask = test_df['survey_id'] == survey_id
    survey_predictions = ensemble_pred[survey_mask]
    
    poverty_rates = calculate_poverty_rates(survey_predictions, poverty_thresholds)
    submission_rates_list.append({
        'survey_id': survey_id,
        **poverty_rates
    })
    print(f"   Survey {survey_id}: {sum(survey_mask)} households")

submission_rates = pd.DataFrame(submission_rates_list)
print(f"\n   Shape: {submission_rates.shape}")

GENERATING SUBMISSION FILES

1. Household Consumption:
   Shape: (103023, 2)
     hhid  cons_ppp17
0  400001   11.202405
1  400002    6.800912
2  400003   10.601008
3  400004   12.189213
4  400005    5.872244

2. Poverty Rates for 3 surveys:
   Survey 400000: 34565 households
   Survey 500000: 34245 households
   Survey 600000: 34213 households

   Shape: (3, 20)


In [18]:
# Save submissions
import os
os.makedirs('submissions', exist_ok=True)

submission_hh.to_csv('submissions/predicted_household_consumption.csv', index=False)
submission_rates.to_csv('submissions/predicted_poverty_distribution.csv', index=False)

print("\nSubmission files saved!")
print("- submissions/predicted_household_consumption.csv")
print("- submissions/predicted_poverty_distribution.csv")

# Also save optimized parameters for reference
import json
with open('submissions/best_params.json', 'w') as f:
    json.dump({
        'xgb': best_xgb_params,
        'lgb': best_lgb_params
    }, f, indent=2)
print("- submissions/best_params.json")


Submission files saved!
- submissions/predicted_household_consumption.csv
- submissions/predicted_poverty_distribution.csv
- submissions/best_params.json


In [19]:
print("="*60)
print("FINAL SUMMARY")
print("="*60)

print(f"\nOptimized XGBoost CV: {study_xgb.best_value:.4f}%")
print(f"Optimized LightGBM CV: {study_lgb.best_value:.4f}%")
print(f"Advanced NN CV: {nn_score:.4f}%")

print(f"\nEnsemble Weights: {weights}")
print(f"Multi-seed averaging: {len(seeds)} seeds")

print("\n" + "="*60)          
print("Ready for submission!")
print("="*60)

FINAL SUMMARY

Optimized XGBoost CV: 27.8692%
Optimized LightGBM CV: 27.8520%
Advanced NN CV: 33.4190%

Ensemble Weights: {'XGBoost': 0.35, 'LightGBM': 0.35, 'CatBoost': 0.2, 'NN': 0.1}
Multi-seed averaging: 5 seeds

Ready for submission!


In [20]:
# NEW ENSEMBLE - WITHOUT NN
weights_clean = {
    'XGBoost': 0.40,
    'LightGBM': 0.40,
    'CatBoost': 0.20
}

ensemble_clean = (
    weights_clean['XGBoost'] * xgb_preds +
    weights_clean['LightGBM'] * lgb_preds +
    weights_clean['CatBoost'] * cat_preds
)

ensemble_clean = np.clip(ensemble_clean, y.min() * 0.5, y.max() * 1.5)

print(f"Clean Ensemble (no NN):")
print(f"  Range: [{ensemble_clean.min():.2f}, {ensemble_clean.max():.2f}]")
print(f"  Mean: {ensemble_clean.mean():.2f}")

# Save clean submission
submission_hh_clean = pd.DataFrame({
    'hhid': test_df['hhid'],
    'cons_ppp17': ensemble_clean
})
submission_hh_clean.to_csv('submissions/predicted_household_consumption_clean.csv', index=False)
print("Saved: submissions/predicted_household_consumption_clean.csv")


Clean Ensemble (no NN):
  Range: [1.73, 112.48]
  Mean: 11.86
Saved: submissions/predicted_household_consumption_clean.csv


In [21]:
# ============================================================
# EXPERIMENT: 3 DIFFERENT ENSEMBLE STRATEGIES
# ============================================================

# Store all versions
submissions = {}

# VERSION 1: Current clean ensemble (40-40-20)
v1_pred = 0.40 * xgb_preds + 0.40 * lgb_preds + 0.20 * cat_preds
v1_pred = np.clip(v1_pred, y.min() * 0.5, y.max() * 1.5)
submissions['v1_balanced'] = v1_pred
print(f"V1 (Balanced 40-40-20): mean={v1_pred.mean():.2f}")

# VERSION 2: LightGBM-heavy (best CV score)
v2_pred = 0.30 * xgb_preds + 0.50 * lgb_preds + 0.20 * cat_preds
v2_pred = np.clip(v2_pred, y.min() * 0.5, y.max() * 1.5)
submissions['v2_lgb_heavy'] = v2_pred
print(f"V2 (LGB-heavy 30-50-20): mean={v2_pred.mean():.2f}")

# VERSION 3: Median ensemble (more robust to outliers)
v3_pred = np.median([xgb_preds, lgb_preds, cat_preds], axis=0)
v3_pred = np.clip(v3_pred, y.min() * 0.5, y.max() * 1.5)
submissions['v3_median'] = v3_pred
print(f"V3 (Median ensemble): mean={v3_pred.mean():.2f}")

# VERSION 4: Geometric mean (good for skewed distributions)
v4_pred = np.exp(np.mean([np.log(xgb_preds + 1), 
                          np.log(lgb_preds + 1), 
                          np.log(cat_preds + 1)], axis=0)) - 1
v4_pred = np.clip(v4_pred, y.min() * 0.5, y.max() * 1.5)
submissions['v4_geometric'] = v4_pred
print(f"V4 (Geometric mean): mean={v4_pred.mean():.2f}")

# Compare distributions with training
print(f"\nTraining stats: mean={y.mean():.2f}, median={np.median(y):.2f}")


V1 (Balanced 40-40-20): mean=11.86
V2 (LGB-heavy 30-50-20): mean=11.86
V3 (Median ensemble): mean=11.85
V4 (Geometric mean): mean=11.85

Training stats: mean=11.56, median=8.77


In [22]:
# ============================================================
# POST-PROCESSING: Match training distribution
# ============================================================

def calibrate_predictions(preds, y_train):
    """Calibrate predictions to match training distribution."""
    # Rank-based calibration
    train_sorted = np.sort(y_train)
    pred_ranks = np.argsort(np.argsort(preds))
    n_train = len(train_sorted)
    n_pred = len(preds)
    
    # Map prediction ranks to training values
    calibrated = train_sorted[(pred_ranks * n_train // n_pred).clip(0, n_train-1)]
    
    return calibrated

# Apply to best version
v1_calibrated = calibrate_predictions(v1_pred, y)
print(f"V1 Calibrated: mean={v1_calibrated.mean():.2f}, std={v1_calibrated.std():.2f}")
print(f"Training:      mean={y.mean():.2f}, std={y.std():.2f}")


V1 Calibrated: mean=11.55, std=10.01
Training:      mean=11.56, std=10.03


In [23]:
# ============================================================
# PREPARE 3 SUBMISSION VERSIONS
# ============================================================
import os
os.makedirs('submissions', exist_ok=True)

# Get poverty thresholds
rate_cols = [c for c in train_rates.columns if c.startswith('pct_hh_below')]
poverty_thresholds = [float(c.replace('pct_hh_below_', '')) for c in rate_cols]

def calculate_poverty_rates(predictions, thresholds):
    rates = {}
    n = len(predictions)
    for threshold in thresholds:
        col_name = f'pct_hh_below_{threshold:.2f}'
        rates[col_name] = (predictions < threshold).sum() / n
    return rates

def create_submission_files(predictions, version_name):
    """Create both submission files for a given prediction."""
    
    # 1. Household consumption
    sub_hh = pd.DataFrame({
        'hhid': test_df['hhid'],
        'cons_ppp17': predictions
    })
    
    # 2. Poverty rates for each survey
    test_surveys = test_df['survey_id'].unique()
    rates_list = []
    for survey_id in test_surveys:
        survey_mask = test_df['survey_id'] == survey_id
        survey_preds = predictions[survey_mask]
        rates = calculate_poverty_rates(survey_preds, poverty_thresholds)
        rates_list.append({'survey_id': survey_id, **rates})
    
    sub_rates = pd.DataFrame(rates_list)
    
    # Save
    sub_hh.to_csv(f'submissions/{version_name}_consumption.csv', index=False)
    sub_rates.to_csv(f'submissions/{version_name}_poverty.csv', index=False)
    
    print(f"✓ {version_name}: consumption shape={sub_hh.shape}, rates shape={sub_rates.shape}")
    return sub_hh, sub_rates

# ============================================================
# CREATE 3 VERSIONS
# ============================================================

print("Creating submission files...\n")

# SUBMIT 1: Calibrated (recommended first try)
create_submission_files(v1_calibrated, "submit1_calibrated")

# SUBMIT 2: Balanced uncalibrated (backup)
create_submission_files(v1_pred, "submit2_balanced")

# SUBMIT 3: Median ensemble (most robust)
create_submission_files(v3_pred, "submit3_median")

print("\n" + "="*50)
print("READY! Files in 'submissions/' folder:")
print("="*50)
print("""
SUBMIT 1 (Recommended first):
  - submit1_calibrated_consumption.csv
  - submit1_calibrated_poverty.csv
  
SUBMIT 2 (If calibrated fails):
  - submit2_balanced_consumption.csv
  - submit2_balanced_poverty.csv
  
SUBMIT 3 (Most robust):
  - submit3_median_consumption.csv
  - submit3_median_poverty.csv
""")


Creating submission files...

✓ submit1_calibrated: consumption shape=(103023, 2), rates shape=(3, 20)
✓ submit2_balanced: consumption shape=(103023, 2), rates shape=(3, 20)
✓ submit3_median: consumption shape=(103023, 2), rates shape=(3, 20)

READY! Files in 'submissions/' folder:

SUBMIT 1 (Recommended first):
  - submit1_calibrated_consumption.csv
  - submit1_calibrated_poverty.csv
  
SUBMIT 2 (If calibrated fails):
  - submit2_balanced_consumption.csv
  - submit2_balanced_poverty.csv
  
SUBMIT 3 (Most robust):
  - submit3_median_consumption.csv
  - submit3_median_poverty.csv

